In [1]:
import os
import ffmpy
from pytube import YouTube
from pytube import Search
import re
import pandas as pd

from youtube_search import YoutubeSearch

In [2]:
import scipy.io.wavfile as wavfile

from dataset_creation import chunks, extract_peaks_and_freqs, final_data_collection
from get_audio_from_link import obtain_youtube_link, delete_spaces, download_audio, remove_audio, from_mp4_to_wav

In [12]:
yt = YouTube('http://youtube.com/watch?v=2lAe1cqCOXo')

In [13]:
yt.length

337

In [4]:
fruit = { 'orange' : [3,2,0,1], 'apple' : [0,3,7,2], 'grapes' : [7,14,6,15] } 
df1 = pd.DataFrame(fruit) 
fruit_2 = { 'orange' : [5,4,3,2], 'apple' : [0,94,3,2], 'grapes' : [3,5,7,8] } 
df2 = pd.DataFrame(fruit_2)

In [10]:
df1

,orange,apple,grapes
0,3,0,7
1,2,3,14
2,0,7,6
3,1,2,15


In [11]:
df2

,orange,apple,grapes
0,5,0,3
1,4,94,5
2,3,3,7
3,2,2,8


In [9]:
pd.concat((df1,df2), axis=0).reset_index(drop=True)

,orange,apple,grapes
0,3,0,7
1,2,3,14
2,0,7,6
3,1,2,15
4,5,0,3
5,4,94,5
6,3,3,7
7,2,2,8


In [35]:
common_path = '/home/jacs/Documents/DataScience/Personal/'

dummy_path = 'song_similarity/'
# As an input there is going to be a list of youtube links that contain audio that can be
# usefull to expand and enrich the database of that particular instrument

# As an input we want a tupple, with the link as first position and instrument as second position

input_path = 'song_similarity_audio/trumpet/'

df_links = pd.read_csv(common_path+input_path+'trumpet_youtube_database_enrichment.csv')

links_audio = list(df_links['youtube_links'])
titles = list(df_links['title'])

range_1 = str(df_links['from'].iloc[0])
range_2 = str(df_links['to'].iloc[0])

print(type(range_1))
print(range_1)
print(type(range_2))
print(range_2)

<class 'str'>
1:30
<class 'str'>
2:30


In [36]:
titles

['database_trumpet_10_peaks_youtube_1']

In [30]:
print(links_audio)

['https://www.youtube.com/watch?v=2nPwkb9ZyPQ']


In [3]:
def min_to_sec(number):
    number = str(number).split(':')
    print(number)
    number = int(number[0])*60 + int(number[1])
    return number

def audio_partition(audio, Fs, range_1, range_2):
    if range_1=='' and range_2=='':
        range_1='0:00'
        range_2= str(audio.shape[0]/Fs).replace('.',':')
    length = audio.shape[0]/Fs
    range_1_index = int(min_to_sec(range_1)*Fs)
    range_2_index = int(min_to_sec(range_2)*Fs)
    audio_cut = audio[range_1_index:range_2_index]
    return audio_cut

In [4]:
for kk in range(0,len(links_audio)):
    linko = links_audio[kk]
    title_file = str(download_audio(linko))
    database_name = str(titles[kk])
    df_final = pd.DataFrame({'peak_1': [], 'peak_2': [], 'Magnitude difference': [],'instrument': [], 'note_played': []})
    print(title_file)
    print(database_name)
    try:
        Fs, audio = wavfile.read(title_file+'.wav')
    except:
        from_mp4_to_wav(title_file+'.mp4',common_path+dummy_path)
        Fs, audio = wavfile.read(title_file+'.wav')
        remove_audio(title_file+'.mp4')
    print(len(audio)-(Fs*60))
    length = audio.shape[0] / Fs
    audio = audio_partition(audio, Fs, range_1, range_2)
    print(len(audio))

NameError: name 'links_audio' is not defined

960000


In [37]:
len(audio)

349237

In [5]:
inputdir = '/home/jacs/Documents/DataScience/Personal/song_similarity/'

In [6]:
def obtain_youtube_link(busqueda, maxim_results):
    results = YoutubeSearch(busqueda, max_results=maxim_results).to_dict()
    linkos_kun = []
    for v in results:
        linkos_kun.append('https://www.youtube.com' + v['url_suffix'])
    return linkos_kun

In [7]:
def download_audio(link):
    selected_video = YouTube(link)
    title = delete_spaces(selected_video)
    audio = selected_video.streams.filter(only_audio=True, file_extension='mp4').first()
    audio.download(filename=title+'.wav')
    return title

In [8]:
def delete_spaces(selected_video):
    title2 = selected_video.title
    title2 = re.sub("(\s)","_"," ".join(re.sub("(-)|(&)|(\[)|(\])","",title2).split()))
    return title2
#re.sub("(\s)|(-)|(&)|(\[)|(\])","_",title, count=0, flags=0)

In [9]:
def remove_audio(title):
    os.remove(title+'.wav')

In [10]:
links = obtain_youtube_link('trumpet solo',10)

In [11]:
links

['https://www.youtube.com/watch?v=2nPwkb9ZyPQ',
 'https://www.youtube.com/watch?v=d70fiI2Mn_A',
 'https://www.youtube.com/watch?v=B1eyZAedhY4',
 'https://www.youtube.com/watch?v=mufSkxC-s3Q',
 'https://www.youtube.com/watch?v=rDw-lY1_1lk',
 'https://www.youtube.com/watch?v=ATajmT5BD6M',
 'https://www.youtube.com/watch?v=_ByGpqf1ldI',
 'https://www.youtube.com/watch?v=h85k-rdiREk',
 'https://www.youtube.com/watch?v=E5Hnnf2M_VM',
 'https://www.youtube.com/watch?v=4pqG3kpSsM8']

In [22]:
for linko in links:
    title = download_audio(linko)
    remove_audio(title)

KeyboardInterrupt: 

In [ ]:
def from_mp4_to_wav(filename, inputdir):
    for filename in os.listdir(inputdir):
        actual_filename = filename[:-4]
        if(filename.endswith(".mp4")):
            os.system('ffmpeg -i {} -acodec pcm_s16le -ar 16000 {}.wav'.format(filename, actual_filename))
        else:
            continue